In [1]:
import open3d as o3d
import numpy as np
import sys

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
sys.path.append('../sample')
from readdatas import *

In [3]:
path = '..\Point Cloud Data\Corner.ply'
pcd =  ReadDatas(path)

In [ ]:
o3d.visualization.draw_geometries([pcd.pcd])

In [ ]:
csf = CsfGroundExtractor()
csf.set_data(pcd)
csf._process_data()

In [ ]:
ran = RansacGroundExtractor()
ran.set_data(pcd)
ran._process_data()

In [4]:
gpf = GpfGroundExtractor()
gpf.set_data(pcd)
gpf._process_data()

In [5]:
ground_o3d = gpf.np_to_o3d(pcd.ground)

In [ ]:
o3d.visualization.draw_geometries([ground_o3d])

In [ ]:
reg = ReGrowSegment(pcd)
reg._process_data()

In [ ]:
dps = DriPathSegment2(pcd)


In [ ]:

dps._process_data()

In [ ]:
dri = DriPathSegment(pcd)
dri._process_data()


In [6]:

path = 'SVM_model.pkl'
svm = SVM(pcd,path)
svm._process_data()


[Open3D INFO] Picked point #4957 (-10., -14., -1.6) to add in queue.
[Open3D INFO] Picked point #19594 (-4.6, -13., -1.6) to add in queue.
[Open3D INFO] Picked point #36077 (-0.58, -11., -1.5) to add in queue.
[Open3D INFO] Picked point #41124 (1.6, -11., -1.5) to add in queue.


In [9]:
side = ground_o3d.select_by_index(pcd.side, invert= True)

In [10]:

no_ground = gpf.np_to_o3d(pcd.no_ground)
# side = pcd.side
paves = pcd.no_side.select_by_index(pcd.paves)
paves.paint_uniform_color([1,0,0])
o3d.visualization.draw_geometries([paves,no_ground,side, pcd.no_side])

In [ ]:
paves = ground_o3d.select_by_index(pcd.paves)
no_paves = ground_o3d.select_by_index(pcd.paves, invert=True)
no_ground = gpf.np_to_o3d(pcd.no_ground)
paves.paint_uniform_color([1,0,0])
o3d.visualization.draw_geometries([no_paves,paves,no_ground])

In [ ]:
paves = pcd.no_side.select_by_index(pcd.paves)
paves.paint_uniform_color([1,0,0])
no_ground = gpf.np_to_o3d(pcd.no_ground)
o3d.visualization.draw_geometries([paves, no_ground, pcd.side, pcd.no_side])

In [ ]:
o3d.visualization.draw_geometries([paves])

In [ ]:
pave = euclidean_cluster(dri,paves, tolerance=0.15)

In [ ]:
ground1 = paves.select_by_index(pave)
o3d.visualization.draw_geometries([ground1])